In [1]:
import os
os.chdir("/home/ubuntu/budget_optimization")
os.getcwd()

'/home/ubuntu/budget_optimization'

In [2]:
import pandas as pd
import requests
from budget_optimization import core

In [52]:
bucket = "lee-budget-optimization"
download_folder = "tmp"
data_file = "OPRI_DATA_NATIONAL.csv"
opri_indicators = {
    "X.US.1.FSGOV": "expenditure_primary",
    "X.US.2T3.FSGOV": "expenditure_secondary",
    "20062": "enrollment_primary",
    "20082": "enrollment_secondary",
    "PRP.1": "private_primary",
    "PRP.2T3": "private_secondary",
}
wb_indicators = {
    "NY.GDP.MKTP.KD.ZG": "gdp_growth",
    "SP.POP.GROW": "population_growth",
}

In [128]:
filepath = f"{download_folder}/{data_file}"
df = (
    pd.read_csv(filepath)
    .rename(str.lower, axis="columns")
    .drop(columns=["magnitude", "qualifier"])
    .loc[lambda df: df["indicator_id"].isin(opri_indicators.keys())]
    .assign(indicator_desc=lambda df: df["indicator_id"].map(opri_indicators))
    .drop(columns="indicator_id")
    .pivot(values="value", index=["country_id", "year"], columns="indicator_desc")
    .rename_axis(None, axis="columns")
    .reset_index()
    .assign(expenditure_primary=lambda df: df["expenditure_primary"] * 1000000)
    .assign(expenditure_secondary=lambda df: df["expenditure_secondary"] * 1000000)
    .assign(private_primary=lambda df: df["private_primary"] / 100)
    .assign(private_secondary=lambda df: df["private_secondary"] / 100)
)
df

,country_id,year,enrollment_primary,enrollment_secondary,expenditure_primary,expenditure_secondary,private_primary,private_secondary
0,ABW,1998,NaN,NaN,1.829781e+07,1.982911e+07,NaN,NaN
1,ABW,1999,9096.0,6159.0,2.040763e+07,2.212884e+07,0.829376,NaN
2,ABW,2000,9263.0,6178.0,2.495123e+07,2.446512e+07,0.824463,NaN
3,ABW,2001,9436.0,6428.0,2.378444e+07,2.689439e+07,0.808923,0.914126
4,ABW,2002,9840.0,6757.0,2.771785e+07,2.677502e+07,0.806809,0.914607
...,...,...,...,...,...,...,...,...
9064,ZWE,2017,2676485.0,NaN,NaN,NaN,0.128468,NaN
9065,ZWE,2018,2725970.0,NaN,NaN,NaN,0.135340,NaN
9066,ZWE,2019,2789692.0,NaN,NaN,NaN,0.129013,NaN
9067,ZWE,2020,2869735.0,NaN,NaN,NaN,0.131401,NaN


In [127]:
dict = {key: [] for key in ["country", "country_id", "year", "value", "indicator_desc"]}
for key in wb_indicators.keys():
    r = requests.get(
        f"http://api.worldbank.org/v2/country/all/indicator/{key}?date=2000:2023&format=json&per_page=30000"
    )
    for x in r.json()[1]:
        dict["country"].append(x["country"]["value"])
        dict["country_id"].append(x["countryiso3code"])
        dict["year"].append(x["date"])
        dict["value"].append(x["value"])
        dict["indicator_desc"].append(wb_indicators[key])
df = (
    pd.DataFrame(dict)
    .pivot(
        values="value",
        index=["country", "country_id", "year"],
        columns="indicator_desc",
    )
    .rename_axis(None, axis="columns")
    .reset_index()
    .assign(gdp_growth=lambda df: df["gdp_growth"] / 100)
    .assign(population_growth=lambda df: df["population_growth"] / 100)
)
df

,country,country_id,year,gdp_growth,population_growth
0,Afghanistan,AFG,2000,NaN,0.014438
1,Afghanistan,AFG,2001,NaN,0.007425
2,Afghanistan,AFG,2002,NaN,0.064493
3,Afghanistan,AFG,2003,0.088323,0.075410
4,Afghanistan,AFG,2004,0.014141,0.039332
...,...,...,...,...,...
5847,Zimbabwe,ZWE,2017,0.040803,0.020436
5848,Zimbabwe,ZWE,2018,0.050099,0.020205
5849,Zimbabwe,ZWE,2019,-0.063324,0.019893
5850,Zimbabwe,ZWE,2020,-0.078170,0.020311
